## How to do an efficient crossmatch of a user table with a table in Data Lab

In this notebook, we demonstrate how to do an efficient crossmatch of a user-provided table of data against a table hosted by Data Lab.  The basic steps are:

* [Make sure that you are logged in as a registered user](#login)
* [Upload your table into myDB, your personal database storage area](#import)
* [Create and run a query to do the crossmatch using the q3c_join() function](#crossmatch)

Click one of the links above if you want to jump to a section.

*Revised: 09/24/18*

<a class="anchor" id="login"></a>
### Standard Imports and Data Lab Login

In [ ]:
from dl import authClient as ac
from dl import storeClient as sc
from dl import queryClient as qc
from dl.helpers.utils import convert        # to use Pandas
import os, getpass

do_cleanup = False                          # Remove any files/tables created by this notebook

# Being logged in as a registered user is a necessary step since we will be using MyDB for storage.  
# Once logged in, the token is stored on the server in the user's $HOME/.datalab directory.  
# Passing the token to queryClient and storeClient is optional.

# Check if logged in as user other than anonymous
try:
    user = ac.whoAmI()
    if (user == 'anonymous'):
        logged_in = False
    else:
        logged_in = True
        token = ac.login(user) #Retrieve token (OPTIONAL)
except:
    logged_in = False

# if not, prompt user to log in
if (not logged_in):
    try:
        token = ac.login(input('Enter user name (+ENTER): '),getpass.getpass('Enter password (+ENTER): '))
    except Exception as e:
        print ('Error: ' + str(e))

print('User: ' + ac.whoAmI())

### Get list of MyDB tables
Let's start by getting a list of existing tables in MyDB. The way to list tables in your MyDB is with the *mydb_list()* method:

        result = queryClient.mydb_list ([token], <table_name>)
where

        token         login token (optional)
        table_name    name of the MyDB table to list (optional)
        
If no arguments are given, a list of all tables in the user MyDB is returned, one tablename per row.  If a table name is provided, the method returns a CSV string on column name and datatype, one column name per row.

In [ ]:
print("Listing mydb tables:\n" + qc.mydb_list())

### Query database with output to mydb
The most basic way to create a table in mydb is to issue a query against a table in the Data Lab database with output to mydb:

In [ ]:
query = 'select top 15 id,raj2000,dej2000 from usno.a2'
try:
    qc.query(adql=query, fmt='csv', out='mydb://usno_objects')
except Exception as e:
    print(e.message)

print("Listing mydb tables:\n" + qc.mydb_list())

## Importing a user table
But often a user will have an existing table that they wish to import into mydb.  The sections below show how to do this with a variety of examples.

### Generate some sample data
First, we'll create a table of data and store it locally as well put in VOSpace.  If you have a table on your local machine, you can use the Jupyter notebook's Upload feature to copy the table into Data Lab.

In [ ]:
data = qc.query ('select id,raj2000 as ra,dej2000 as dec from usno.a2 limit 15')  # CSV String
with open ('objects.csv','w') as fd:                                               # file of CSV data
    fd.write (data)   
df = convert(data)                                                                 # Pandas DataFrame
tsv = data.replace(',','\t')                                                       # Tab-separated
asv = data.replace(',',' ')                                                        # ASCII-separated
bsv = data.replace(',','|')                                                        # Bar-separated

if sc.access ('vos://objects.csv'):
    sc.rm ('vos://objects.csv')
if sc.access ('vos://objects.vot'):
    sc.rm ('vos://objects.vot')
sc.put ('objects.csv', 'vos://objects.csv')                                        # Put a copy in VOSpace
print(sc.ls ('objects.*', format='long', verbose=True))

We could also have output the result of the query above directly to VOSpace.

In [ ]:
query = 'select top 15 id,raj2000,dej2000 from usno.a2'
qc.query(adql=query, fmt='votable', out='vos://objects.vot') # FIXME - query save to existing file fails
print(sc.ls ('objects.*', format='long', verbose=True))

<a class="anchor" id="import"></a>
### Import a Table Directly to MyDB
The simplest way to import a table to your MyDB is with the *mydb_import()* method:

        result = queryClient.mydb_import ([token], <table_name>, <data> [, schema=<schema_def>])
where

        token         login token (optional)
        table_name    name of the MyDB table to create
        data          filename of CSV data, a CSV string, or a Pandas DataFrame object
        schema_def    [Optional, default=""] The schema definition is stored in a text file or string. 
                      This is a CSV-formatted file that contains column name, (Postgres) data 
                      type, one row per column.
        drop          [Optional, default=True] Drop any existing table before loading the new one.

#### Example 1: Import a (local) CSV file to a MyDB table called 'objects1'

In [ ]:
print ('Local file result: '   + qc.mydb_import ('objects1', 'objects.csv'))

#### Example 2: Import a CSV file from your Virtual Storage to a MyDB table called 'objects2'

In [ ]:
print ('VOS file result: '   + qc.mydb_import ('objects2', 'vos://objects.csv', verbose=True))

#### Example 3:  Import directly from a string containing CSV (or other delimited) data

In [ ]:
print ('String (CSV) result: ' + qc.mydb_import ('objects3', data))
#print ('String (TSV) result: ' + qc.mydb_import ('objects3', data, delimiter='\t'))
#print ('String (ASV) result: ' + qc.mydb_import ('objects3', data, delimiter=' '))
#print ('String (BSV) result: ' + qc.mydb_import ('objects3', data, delimiter='|'))

#### Example 4: Import a Pandas DataFrame directly to a MyDB table

In [ ]:
print ('Pandas result: ' + qc.mydb_import ('objects4', df))

#### Example 5:  Import CSV data, then append to the table

In [ ]:
res = qc.mydb_import ('objects5', data, verbose=True)             # Drop existing table by default
print ('Num rows in objects5: ' + qc.query('select count(*) from mydb://objects5',fmt='csv'))

res = qc.mydb_import ('objects5', data, verbose=True, drop=False) # Disable 'drop' to append
print ('Num rows in objects5: ' + qc.query('select count(*) from mydb://objects5',fmt='csv'))

#### Example 6:  Import a VOTable stored in Virtual Storage

In [ ]:
# Generate a VOTable by calling the PanSTARRS Cone Search service.
ps1_base_url = 'http://gsss.stsci.edu/webservices/vo/ConeSearch.aspx?CAT=PS1V3OBJECTS&'
ps1_url = ps1_base_url + ('RA={0}&DEC={1}&SR={2}'.format(0.0,0.0,0.05))
os.system ('wget -q -O ps1.vot "' + ps1_url + '"')

# Put a copy in virtual storage.
sc.put('ps1.vot',to='vos://ps1.vot')

print (ps1_url)

Now load the MyDB table directly from the VOTable on local disk, in VOSpace, and directly from the URL:

In [ ]:
res = qc.mydb_import ('objects6a', 'ps1.vot', verbose=True)             # Load from local VOTable
print ('import res: ' + res)
print ('Num rows in objects6a: ' + qc.query('select count(*) from mydb://objects6a',fmt='csv'))

In [ ]:
res = qc.mydb_import ('objects6b', 'vos://ps1.vot', verbose=True)       # Load from virtual storage VOTable
print ('Num rows in objects6b: ' + qc.query('select count(*) from mydb://objects6b',fmt='csv'))

In [ ]:
res = qc.mydb_import ('objects6c', ps1_url, verbose=True)               # Load from service URL directly
print ('Num rows in objects6c: ' + qc.query('select count(*) from mydb://objects6c',fmt='csv'))

Note that for each of the tables created, the column names and data types were determined automatically.  We can check whether these are correct by listing the columns using *mydb_list()*.

In [ ]:
print (qc.mydb_list('objects4'))              # Use of the 'mydb://' prefix is optional
print (qc.mydb_list('mydb://objects6a'))

***
## Piecewise Loading of MyDB Tables

Importing data to MyDB requires two basic steps:  1) Creating the table in the database, and 2) loading the data.  The *mydb_import()* method combines these steps as a convenience but provides additional functionality to automatically convert data formats and derive the table schema.  If you should need to override these steps for some reason, each step can be done individually.

### Create the MyDB table
Creating an empty MyDB table is done with the *mydb_create()* method:

    result = queryClient.mydb_create ([token], <table_name>, <schema_def>)
where

    token         login token (optional)
    table_name    name of the MyDB table to create
    schema_def    filename containing a schema definition, or a string containing the schema definition
    drop          [Optional, default=True] Drop any existing table before loading the new one.
    
The *mydb_create()* step needs a table name (needs to not duplicate an existing myDB table) and a schema definition.  The schema definition is stored in a text file, in this case in the user notebook directory. The schema definition file is a CSV-formatted file that contains column name and (Postgres) data type, one row per column. The token here is optional.

In [ ]:
# Create a simple (id,ra,dec) schema of a text string and two double values
schema_str = 'id,text\nra,double precision\ndec,double precision\n'
with open ('schema.txt','w') as fd:
    fd.write (schema_str)

In [ ]:
# Example 1:  Create a table from a schema definition file, drop existing table
print("Creating table 'test'... ",qc.mydb_create('test1','schema.txt',drop=True))

In [ ]:
# Example 2:  Create a table from a schema definition str
print("Creating table 'test'... ",qc.mydb_create('test2', schema_str, drop=True))

In [ ]:
print("table1 structure:\n" + qc.mydb_list('test1'))
print("table2 structure:\n" + qc.mydb_list('test2'))

### Insert data 
Loading a MyDB table with data is done with the *mydb_insert()* method:

    result = queryClient.mydb_insert ([token], <table_name>, <data>)
where

    token         login token (optional)
    table_name    name of the MyDB table to load
    data          filename of CSV data or a CSV string
    csv_header    [Optional, default=True] CSV data contains a header row of column names
    
Insert the data from a file/string in user's notebook space to the newly created table in myDB. 

In [ ]:
# Example 1:  Load from a CSV file
print('Loading from file:   ' + qc.mydb_insert('test1', 'objects.csv'))

# Example 2: Load from a CSV string
print('Loading from string: ' + qc.mydb_insert('test2', data))

### Retrieve data from myDB table to verify
You can use queryClient to query tables in mydb, exactly as you would for Data Lab database tables.

In [ ]:
print("Running query: 'select * from mydb://test1'")
df1=convert(qc.query(sql="select * from mydb://test1"))
df1

<a class="anchor" id="crossmatch"></a>
## Run the crossmatch
We now construct a query to run the crossmatch in the database using *q3c_join()*, which identifies all matching objects within a specified radius. (See https://github.com/segasai/q3c for details on using Q3C functions.)  The format is 

        q3c_join (ra_user, dec_user, ra_DLtable, dec_DLtable, radius) 
where it's important that the small user catalog appear FIRST in the query, and the large table from Data Lab database SECOND. The radius is specified in DEGREES.  The user table is referred to using a *mydb://* prefix on the table name. In this example, we parse the result into a Pandas dataframe with the DL helpers module *convert()*.  In the query, we retrieve just the user ID and the object designation from Gaia DR2 along with the separating distance by using the *q3c_dist()* function (and convert the return value in degrees to arcsec).

If you find your q3c_join query runs a long time, check:
* That the ra, dec of the SMALL catalog appears FIRST in the query
* That the radius is in DEGREES, not e.g. arcseconds

(Note that in this notebook, we use a radius of 0.01 degrees or ~36 arcseconds.  It's usually best to start with a smaller radius, especially if you expect a lot of matches.)

In [ ]:
%%time

# xmatch query of mydb://objects6a (our small PS1 table) against gaia_dr2.gaia_source within 0.01 degrees, return all matches
sql = '''SELECT  o.objid, g.designation,
                (q3c_dist(o.ramean,o.decmean,g.ra,g.dec)*3600.0) as dist_arcsec
         FROM mydb://objects6a AS o, 
              gaia_dr2.gaia_source AS g 
         WHERE q3c_join(o.ramean, o.decmean, g.ra, g.dec, 0.01)'''
df2 = convert(qc.query(sql=sql))

In [ ]:
print ('nrows x ncols: ' + str(df2.shape))
df2[0:10] # first 10 rows, note multiple matches

Note again that this query returns *__all__* matches within the radius for each object in list.  The crossmatch query can be written to other result tables such as only the nearest neighbor, only those objects in the user table that have matches, non-matches (i.e. objects with no match the find catalog dropouts), or it can be used to pre-filter either table (e.g. only match 'red' objects with a stellar profile).  The Q3C documentation (see https://github.com/segasai/q3c) contains additional example queries you can try.

__Want to do just a nearest neighbor search?__  This example adapted from the Q3C documentation picks just the nearest neighbor, returning null if none are found:

In [ ]:
%%time

sql = '''SELECT  o.objid, gg.designation, (q3c_dist(o.ramean,o.decmean,gg.ra,gg.dec)*3600.0) as dist_arcsec 
         FROM mydb://objects6a AS o
         LEFT JOIN LATERAL (
               SELECT g.* 
                    FROM 
                        gaia_dr2.gaia_source AS g
                    WHERE
                        q3c_join(o.ramean, o.decmean, g.ra, g.dec, 0.01)
                    ORDER BY
                        q3c_dist(o.ramean,o.decmean,g.ra,g.dec)
                    ASC LIMIT 1
               ) as gg ON true;'''

df3 = convert(qc.query(sql=sql))

In [ ]:
print ('nrows x ncols: ' + str(df3.shape))
df3[0:10] # first 10 rows

__Don't want the non-matching rows?__  Just replace the LEFT JOIN above with an INNER JOIN:

In [ ]:
%%time

sql = '''SELECT  o.objid, gg.designation, (q3c_dist(o.ramean,o.decmean,gg.ra,gg.dec)*3600.0) as dist_arcsec 
         FROM mydb://objects6a AS o
         INNER JOIN LATERAL (
               SELECT g.* 
                    FROM 
                        gaia_dr2.gaia_source AS g
                    WHERE
                        q3c_join(o.ramean, o.decmean, g.ra, g.dec, 0.01)
                    ORDER BY
                        q3c_dist(o.ramean,o.decmean,g.ra,g.dec)
                    ASC LIMIT 1
               ) as gg ON true;'''

df4 = convert(qc.query(sql=sql))

In [ ]:
print ('nrows x ncols: ' + str(df4.shape))
df4[0:10] # first 10 rows

### Cleanup
If the myDB table was meant to be temporary, it can be dropped with *mydb_drop()*.  Here, we'll drop only the tables created in this notebook:

In [ ]:
# Drop any tables or temp files we created.
if do_cleanup:
    for table in ['test1','test2','objects1','objects2','objects3','objects4','objects5','objects6a','objects6b','objects6c']:
        print("Dropping table '" + table + "' ... \t" + qc.mydb_drop(table))
    print("Listing mydb tables:\n" + qc.mydb_list())
    
    for f in ['objects.csv','schema.txt']:
        if os.path.exists(f):
            os.remove (f)
    if sc.access ('vos://objects.csv'):
        print ('removing objects.csv')
        sc.rm ('vos://objects.csv')
    if sc.access ('vos://objects.vot'):
        print ('removing objects.vot')
        sc.rm ('vos://objects.vot')

If instead you want to drop __all__ tables in your MyDB, you can do this by processing the list, e.g.

In [ ]:
if True:      # Change this to remove all tables
    print("Initial listing of mydb tables:\n" + qc.mydb_list())
    table_list = qc.list().split('\n')    # convert string return to array
    for table in table_list:
        print("Dropping table '" + table + "' ... \t" + qc.mydb_drop(table))
    print("\nFinal listing of mydb tables:\n" + qc.mydb_list())
    print("\nFinal list of VOSpace:\n" + sc.ls(format='long'))